In [33]:
import warnings
warnings.filterwarnings("ignore")
# imports
from math import sqrt, log
import gym
import copy
import numpy as np
import gym_go
from gym_go.gogame import areas

In [34]:
# Global constants
UCB_C = 2

# Monte Carlo Three Search

1. Selection
    - Taverse the tree to find greatest UCB-score
2. Expansion
    - If the selected leaf node has been visited before expand by adding weighted game action
3. Rollout
    - Simulate the game until end-condition from the expanded leaf
4. Back-propagation
    - Updating the value of each ancestor node of the expanded leaf


In [35]:
def get_legal_move(env):
    board_shape = env.state().shape[1:]
    pass_id = np.prod(board_shape)
    action = env.action_space.sample() # pick random action
    action2d = action // board_shape[0], action % board_shape[1], action
    while action2d[2] != pass_id and env.state()[3, action2d[0], action2d[1]] == 1:
        action = env.action_space.sample() # pick random action
        action2d = action // board_shape[0], action % board_shape[1], action
    return action2d[2]

def do_other_players_turn(env, colour):
    if env.done:
        return
    board_shape = env.state().shape[1:]
    pass_id = np.prod(board_shape)

    best_move = pass_id
    black_area, white_area = areas(env.state())
    for action in range(0, pass_id + 1):
        action2d = action // board_shape[0], action % board_shape[1]
        if action == pass_id or env.state()[3, action2d[0], action2d[1]] == 0:
            copy_env : gym.Env = copy.deepcopy(env)
            copy_env.step(action)
            step_black_area, step_white_area = areas(copy_env.state())
            if colour == "white" and step_black_area > black_area:
                black_area = step_black_area
                best_move = action
            if colour == "black" and step_white_area > white_area:
                white_area = step_white_area
                best_move = action

    env.step(best_move)

class Node():
    def __init__(self, env, parent):
        self.env : gym.Env = env
        self.value : int = 0 # Value estimate
        self.trials : int = 0 # Number of trials for this node
        self.parent : Node = parent # Parent node of this node
        self.children : list[Node] = [] # List of children of this node
    
    # calculate a Upper Confidence Bound
    def ucb(self, total_trials):
        return self.value + ( UCB_C * sqrt(log(total_trials) / self.trials) )
    
    # Add a new node to a leaf node
    def expansion(self):
        if self.env.done:
            return
        board_shape = self.env.state().shape[1:]
        pass_id = np.prod(board_shape)
        for action in range(0, pass_id + 1):
            action2d = action // board_shape[0], action % board_shape[1]
            if action == pass_id or self.env.state()[3, action2d[0], action2d[1]] == 0:
                child_env = copy.deepcopy(self.env)
                child_env.step(action)
                self.children.append(Node(child_env, self))

    # Simulate game from current move until end-condition returning the score
    def rollout(self):
        if self.env.done:
            return self.env.reward()
        
        rollout_env = copy.deepcopy(self.env)
        rollout_result = 0
        done = False
        while not done:
            random_action = get_legal_move(rollout_env)
            _, reward, done, _ = rollout_env.step(random_action)
            rollout_result += reward
        return rollout_result

class Monte_Carlo_Tree_Search():
    def __init__(self, colour):
        self.env = gym.make('gym_go:go-v0', size=3, komi=0, reward_method='heuristic')
        self.env.reset()
        self.number_of_trials : int = 0
        self.root = Node(self.env, None)
        self.colour = colour
    
    # Update scores of all parent nodes after rollout
    def back_propagation(self, rollout_node: Node, rollout_result):
        current_node = rollout_node
        while current_node != None:
            current_node.trials += 1
            if self.colour == "white":
                current_node.value += -rollout_result
            else: 
                current_node.value += rollout_result
            current_node = current_node.parent
        self.number_of_trials += 1
    
    # find and return the leaf node with the highest UCB-score 
    def selection(self, starting_node: Node):
        selected_child = starting_node
        current_best_ucb = 0
        current_node = starting_node
        while len(current_node.children) > 0:
            for child in current_node.children:
                if child.trials == 0:
                    return child
                if child.ucb(self.number_of_trials) > current_best_ucb:
                    selected_child = child
            current_node = selected_child
            current_best_ucb = 0
            
        return selected_child

    def run(self):
        if self.colour == "white":
            do_other_players_turn(self.root.env, self.colour)
        selected_node = self.root
        selected_node.expansion()
        selected_node = self.root.children[0]

        run = 0
        while not selected_node.env.done:
            selected_node = self.selection(self.root)

            if selected_node.trials > 0:
                selected_node.expansion()
                selected_node = selected_node.children[0]
            else:
                do_other_players_turn(selected_node.env, self.colour)
                if selected_node.env.done:
                    self.back_propagation(selected_node, selected_node.env.reward())
                    self.selection(self.root)
                    run += 1
                    continue

            rollout_result = selected_node.rollout()
            self.back_propagation(selected_node, rollout_result)

            run += 1
            if run > 100:
                return self.selection(self.root)
        return selected_node


In [36]:
model = Monte_Carlo_Tree_Search("black")
best_node = model.run()

best_game = []
current_node = best_node
while current_node != None:
    best_game.append(current_node)
    current_node = current_node.parent

for node in reversed(best_game):
    node.env.render()


other player ended the game on their turn
	0 1 2 
0	╔═╤═╗
1	╟─┼─╢
2	╚═╧═╝
	Turn: BLACK, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 0, White Area: 0

	0 1 2 
0	●═╤═╗
1	╟─┼─╢
2	○═╧═╝
	Turn: BLACK, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 1, White Area: 1

	0 1 2 
0	●═●═╗
1	╟─┼─○
2	○═╧═╝
	Turn: BLACK, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 2, White Area: 2

	0 1 2 
0	●═●═●
1	╟─┼─○
2	○═╧═╝
	Turn: BLACK, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 2, White Area: 3

	0 1 2 
0	●═●═●
1	╟─○─○
2	○═●═╝
	Turn: BLACK, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 3, White Area: 4

	0 1 2 
0	●═●═●
1	●─○─○
2	╚═●═╝
	Turn: BLACK, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 2, White Area: 6

	0 1 2 
0	╔═╤═╗
1	╟─○─○
2	○═●═╝
	Turn: WHITE, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 7, White Area: 1

	0 1 2 
0	╔═╤═╗
1	╟─○─○
2	○═●═╝
	Turn: WHITE, Game State (ONGOING|PASSED|END): END
	Black Area: 7, White Area: 1

